In [1]:
# python3.5及之前，字典是hash算法来实现的
# python3.6以后，字典保留了添加时的顺序
baby_names = {
    'cat': 'kitten',
    'dog': 'puppy',
}
print(baby_names)

{'cat': 'kitten', 'dog': 'puppy'}


- 很容易定义和标准dict相似的实例对象
- 不能假设迭代时是按照插入时的顺序一致
- 如果想按照标准dict来处理，有三种方案
    - 不依赖插入顺序来编写代码，重写排序规则
    - 运行时判断是不是标准dict
    - 给代码加类型注解并做静态分析

In [2]:
votes = {
    'otter': 1281,
    'polar bear': 587,
    'fox': 863,
}

def populate_ranks(votes, ranks):
    names = list(votes.keys())
    names.sort(key=votes.get, reverse=True) # key其实是指定一种idx的比较规则，可以不涉及对象本身
    for i, name in enumerate(names, 1):
        ranks[name] = i

def get_winner(ranks):
    return next(iter(ranks))

ranks = {}
populate_ranks(votes, ranks)
print(ranks)
winner = get_winner(ranks)
print(winner)

{'otter': 1, 'fox': 2, 'polar bear': 3}
otter


In [3]:
from collections.abc import MutableMapping

class SortedDict(MutableMapping):
    def __init__(self):
        self.data = {}

    def __getitem__(self, key):
        return self.data[key]

    def __setitem__(self, key, value):
        self.data[key] = value

    def __delitem__(self, key):
        del self.data[key]

    def __iter__(self):
        keys = list(self.data.keys())
        keys.sort()
        for key in keys:
            yield key

    def __len__(self):
        return len(self.data)

my_dict = SortedDict()
my_dict['otter'] = 1
my_dict['cheeta'] = 2
my_dict['anteater'] = 3
my_dict['deer'] = 4

# 测试一下数据加载是否正确
assert my_dict['otter'] == 1

assert 'cheeta' in my_dict
del my_dict['cheeta']
assert 'cheeta' not in my_dict

expected = [('anteater', 3), ('deer', 4), ('otter', 1)]
assert list(my_dict.items()) == expected

assert not isinstance(my_dict, dict)

# 构造一个类似标准dict的字典
sorted_ranks = SortedDict()
populate_ranks(votes, sorted_ranks)
print(sorted_ranks.data)
winner = get_winner(sorted_ranks)
print(winner)
# 函数返回按照字母排序顺序，而不是populate_ranks是的插入顺序
# {'otter': 1, 'fox': 2, 'polar bear': 3}
# fox

{'otter': 1, 'fox': 2, 'polar bear': 3}
fox


In [4]:
# 处理方式一:不依赖插入顺序，重写迭代
def get_winner(ranks):
    for name, rank in ranks.items():
        if rank == 1:
            return name

winner = get_winner(sorted_ranks)
print(winner)

otter


In [6]:
# 处理方式二:判断是不是想要的dict
import logging
try:
    def get_winner(ranks):
        if not isinstance(ranks, dict):
            raise TypeError('must provide a dict instance')
        return next(iter(ranks))
    
    assert get_winner(ranks) == 'otter'
    
    get_winner(sorted_ranks)
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "C:\Users\SAT\AppData\Local\Temp\ipykernel_28596\1719194662.py", line 11, in <module>
    get_winner(sorted_ranks)
  File "C:\Users\SAT\AppData\Local\Temp\ipykernel_28596\1719194662.py", line 6, in get_winner
    raise TypeError('must provide a dict instance')
TypeError: must provide a dict instance


In [8]:
# xx.py
# 解决方法三:类型注解
# 使用mypy运行，python -m mypy --strict xx.py
# 类型不符会报错
from typing import Dict, MutableMapping

def populate_ranks(votes: Dict[str, int],
                   ranks: Dict[str, int]) -> None:
    names = list(votes.keys())
    names.sort(key=votes.get, reverse=True)
    for i, name in enumerate(names, 1):
        ranks[name] = i

def get_winner(ranks: Dict[str, int]) -> str:
    return next(iter(ranks))

from typing import Iterator, MutableMapping

class SortedDict(MutableMapping[str, int]):
    def __init__(self) -> None:
        self.data: Dict[str, int] = {}

    def __getitem__(self, key: str) -> int:
        return self.data[key]

    def __setitem__(self, key: str, value: int) -> None:
        self.data[key] = value

    def __delitem__(self, key: str) -> None:
        del self.data[key]

    def __iter__(self) -> Iterator[str]:
        keys = list(self.data.keys())
        keys.sort()
        for key in keys:
            yield key

    def __len__(self) -> int:
        return len(self.data)

votes = {
    'otter': 1281,
    'polar bear': 587,
    'fox': 863,
}

sorted_ranks = SortedDict()
populate_ranks(votes, sorted_ranks)
print(sorted_ranks.data)
winner = get_winner(sorted_ranks)
print(winner)

{'otter': 1, 'fox': 2, 'polar bear': 3}
fox


In [10]:
# 类型注解不用专门的分析工具不会报错
def add(a:int, b:int) -> int:
    return a+b

print(add("1", "2")) # 这样不会报错
print(add(1, 2))

12
3
